In [1]:
import TableParser
import pandas as pd
import tempfile, zipfile
import os

In [2]:
class TestTableParser(TableParser.TableParserItem):
    def __init__(self):
        pass
    
    def parse(self, series: pd.Series):
        mistake = {
            'Jan':'01','Feb': '02','Mar':'03','Apr':'04','May':'05','Jun':'06','Jul':'07','Aug':'08','Sept':'09','Oct':'10','Nov':'11','Dec':'12',
            'o': '0','l': '1','b': '6','g': '9','q': '9','t': '7','v': '5','G': '6','F': '7','Z': '2','Q': '2','B': '8','O': '0','D': '0','E': '3','A': '4','S': '5',
            r'[ -/]|[:-@]|[\[-_]|[{-~]': '-',
        }
        for key, val in mistake.items():
            series = series.str.strip().str.replace(key, val, regex=True)
        return (pd.to_datetime(series, errors="ignore"), [])

    def fix_xlsx(self, in_file):
        tmpfd, tmp = tempfile.mkstemp(dir=os.path.dirname(in_file))
        os.close(tmpfd)
        filename = '[Content_Types].xml'
        data = ''
        with zipfile.ZipFile(in_file, 'r') as zin:
            with zipfile.ZipFile(tmp, 'w') as zout:
                for item in zin.infolist():
                    if item.filename != filename:
                        zout.writestr(item, zin.read(item.filename))
                    else:
                        data = zin.read(filename).decode()
        os.remove(in_file)
        os.rename(tmp, in_file)
        data = data.replace('/xl/sharedStrings.xml', '/xl/SharedStrings.xml')
        with zipfile.ZipFile(in_file, mode='a', compression=zipfile.ZIP_DEFLATED) as zf:
            zf.writestr(filename, data)

    ###############################################
    # to find the x,y coordinate of row with date #
    
    def find_coordinate(self, x, y,df):
        test_list =[[0,0]]
        count_X_axis,count_Y_axis,main_axis = 0,0,"Y axis"
        
        for i in range(1,df.shape[0]):
            if len(test_list) == 11: 
                break
            for j in range(df.shape[1]):
                if type(df.iloc[i,j]) == type("string") and len(df.iloc[i,j]) > 5:
                    if type(testTableParser.parse(pd.Series([df.iloc[i,j]]))[0][0]) == type(pd.to_datetime('01.01.2021')):
                        test_list.append([i,j])
                        if test_list[-1][1] > test_list[-2][1]:
                            count_X_axis += 1
                        if test_list[-1][0] > test_list[-2][0]:
                            count_Y_axis += 1  
                        if len(test_list) == 11:
                            break
        if count_X_axis > count_Y_axis:
            main_axis = "X axis"
#         print(main_axis,test_list)
        return test_list[1][0],test_list[1][1],main_axis

    def all_action(self,df):
        data_row_num, data_col_num,main_axis = self.find_coordinate(0,0,df) # <-- find coordinate of dates
        if main_axis == "X axis":
            table = df.iloc[data_row_num:, data_col_num:].reset_index(drop=True)  # <--- all useful data
            date = df.iloc[data_row_num, data_col_num:]  # <--- Series with date (01.03.2016;...)
            date = self.parse(date)
            table.iloc[0] = date[0]
        elif main_axis == "Y axis":
            table = df.iloc[data_row_num:, data_col_num:].reset_index(drop=True)  # <--- all useful data
            date = df.iloc[data_row_num:,data_col_num]
            date = self.parse(date)
            table.iloc[0:,0] = date[0]

#         table.columns = table.iloc[0] # <--- change column names
#         table = table.iloc[1:, :] # <--- remove first row
        return table

In [6]:
testTableParser = TestTableParser()
# testTableParser.fix_xlsx('file1.xlsx')
df = pd.read_excel("file33.xlsx")
testTableParser.all_action(df)

,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,...,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29
0,2016-01-03 00:00:00,24490,98,10,0.40,0.04,2900.00,24490,117.0,11841.6,...,875,20.20,0.68,21.00,18.93,75.00,67.62,2.018,75.00,65.6
1,2016-02-03 00:00:00,24486,98,4,0.40,0.02,2860.00,24486,117.0,11680.1,...,875,20.20,0.69,21.00,19.35,75.00,69.10,2.018,75.00,65.6
2,2016-03-03 00:00:00,24474,98,12,0.40,0.05,2900.00,24474,117.0,11849.3,...,874,20.20,0.68,21.00,18.95,75.00,67.66,2.018,75.00,65.6
3,2016-04-03 00:00:00,24470,122,4,0.50,0.02,2860.00,24470,117.0,11687.8,...,874,18.10,0.67,19.00,18.74,67.86,66.94,2.018,75.00,65.6
4,2016-05-03 00:00:00,24469,122,1,0.50,NaN,2880.00,24469,117.0,11770.0,...,874,18.10,0.67,19.00,18.74,67.86,66.94,2.018,75.00,65.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
919,2018-10-12 00:00:00,29999,150,10,0.50,0.03,NaN,29999,117.0,NaN,...,1071,18.10,0.73,19.00,16.80,67.86,60.00,1.930,80.00,NaN
920,2018-11-12 00:00:00,29995,150,4,0.50,0.01,NaN,29995,117.0,NaN,...,1071,18.10,0.71,19.00,16.13,67.86,57.61,1.930,80.00,NaN
921,2018-12-12 00:00:00,18092,90,5,0.50,0.03,NaN,18092,117.0,NaN,...,646,18.10,0.65,19.00,24.51,67.86,87.55,1.930,80.00,NaN
922,2018-12-13 00:00:00,7119,107,7,1.50,0.10,NaN,7119,117.0,NaN,...,254,17.00,0.29,18.00,27.61,64.29,98.61,1.930,80.00,NaN


In [16]:
testTableParser.parse(pd.Series(['01.02.2002']))[0][0]

Timestamp('2002-01-02 00:00:00')

In [14]:
# pd.to_datetime('01.02.2002')
df.shape

(30, 927)